### 0. Set up environment for IXI

In [1]:
import os

ROOT_FOLDER = os.path.join(os.path.expanduser('~'), 'data', 'IXI')
NUM_THREADS = 8
MNI152_TEMPLATE = '/usr/local/fsl/data/standard/MNI152_T1_1mm.nii.gz'
IXI_URL = 'http://biomedic.doc.ic.ac.uk/brain-development/downloads/IXI'

tar_path = os.path.join(ROOT_FOLDER, 'IXI-T1.tar')
raw_folder = os.path.join(ROOT_FOLDER, 'images')
freesurfer_fsl_folder = os.path.join(ROOT_FOLDER, 'freesurfer+fsl')
labels_file = os.path.join(ROOT_FOLDER, 'IXI.xls')

ixi_images_url = f'{IXI_URL}/IXI-T1.tar'
ixi_labels_url = f'{IXI_URL}/IXI.xls'

if not os.path.isdir(ROOT_FOLDER):
    print(f'Creating root folder {ROOT_FOLDER}')
    os.mkdir(ROOT_FOLDER)

### 1. Download IXI

In [2]:
import math
import requests
import tarfile

from shutil import rmtree
from tqdm import tqdm

def download(url: str, filename: str, chunksize: int = 2**16):
    """ Downloads a file from a given url to a given path in chunks. """
    resp = requests.get(url, stream=True)

    size = int(resp.headers.get('content-length')) \
           if 'content-length' in resp.headers else None
    
    with open(filename, 'wb') as f:
        downloaded = 0
        for chunk in tqdm(resp.iter_content(chunk_size=chunksize), total=math.ceil(size / chunksize)):
            f.write(chunk)

# Checks whether the folder with raw T1 data exists and contains the correct number of images
if not (os.path.isdir(raw_folder) and len(os.listdir(raw_folder)) == 581):
    print('Downloading and unpacking IXI dataset')
    rmtree(raw_folder)
    os.mkdir(raw_folder)
    
    if not os.path.isfile(tar_path):
        print('Downloading tarball')
        download(ixi_images_url, tar_path)
        
    with tarfile.open(tar_path) as tarball:
        tarball.extractall(raw_folder)

if not os.path.isfile(labels_file):
    print('Downloading IXI labels')
    download(ixi_labels_url, labels_file)

### 2. Perform FreeSurfer+FSL preprocessing

In [3]:
import subprocess
import numpy as np

from threading import Thread


preprocessing_script = os.path.join(os.pardir, 'preprocessing', 'freesurfer_and_fsl.sh')

if not os.path.isfile(preprocessing_script):
    raise ValueError('Unable to find FreeSurfer+FSL preprocessing script')

class Worker(Thread):
    def __init__(self, files, source, destination):
        self.files = files
        self.source = source
        self.destination = destination

        super().__init__()

    def run(self):
        for filename in tqdm(self.files):
            subject = filename.replace('.nii.gz', '')
            cmd = ['bash', preprocessing_script, 
                   '-f', os.path.join(self.source, filename), 
                   '-d', os.path.join(self.destination, subject),
                   '-t', MNI152_TEMPLATE]
            subprocess.call(cmd, stdout=open(os.devnull, 'wb'))

if not os.path.isdir(freesurfer_fsl_folder):
    os.mkdir(freesurfer_fsl_folder)

images = os.listdir(raw_folder)
batches = np.array_split(images, NUM_THREADS)

threads = [Worker(batch, raw_folder, freesurfer_fsl_folder) for batch in batches]
[worker.start() for worker in threads]
[worker.join() for worker in threads]

  0%|                                                                                                                             | 0/73 [00:00<?, ?it/s]

  0%|                                                                                                                             | 0/73 [00:00<?, ?it/s]



  0%|                                                                                                                             | 0/73 [00:00<?, ?it/s]


  0%|                                                                                                                             | 0/73 [00:00<?, ?it/s]




  0%|                                                                                                                             | 0/72 [00:00<?, ?it/s]





  0%|                                                                                                                             | 0/72 [00:00<?, ?it/s]






 19%|██████████████████████                            

KeyboardInterrupt: 